In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
sources = {
    "users" : ['id', "username", "email", 'organization', 'date_joined'],
    "projects": ['id', 'name', 'description', 'user_id', 'status_code', 'public', 'frag_sample_id', 'frag_sim_conf_id', 'frag_compare_conf_id', 'depth_total', 'depth_last_match'],
    "projects_info" : None,
    "fragsamples": ['id', 'user_id', 'name', 'ion_charge', 'ions_total', 'status_code'],
    "fragsamples_info" : None,
    "reactions" : ['id', 'name', 'description', 'user_id', 'reactants_number', 'status_code'],
}
dfs = {}
for name, fields in sources.items():
    path = Path().resolve() / "{}.json".format(name)
    dfs[name] = pd.read_json(path)
    if fields is not None:
        dfs[name] = dfs[name].loc[:,fields]

for name in ("projects", "fragsamples"):
    df_name =  dfs.pop("{}_info".format(name))
    dfs[name] = dfs[name].merge(df_name, on="id", how="outer")

In [ ]:
for name, df in dfs.items():
    print(name)
    print(list(df.columns))

In [ ]:
for name, df in dfs.items():
    if "description" in df.columns:
        print("{} with description : {:.0%}".format(name, len(df.description.unique())/len(df)))

In [ ]:
aggs = {
    "projects": { "matching_count": ("sum", "mean"), "reactions_count": "mean"},
    "fragsamples": {'ions_total':("sum", "mean"), "annotations_count":("sum", "mean") },
    "reactions" : {},
}

dfs_ = dfs.copy()
agg = {label:("first", ) for label in ("organization", "username", "email")}
df_u = dfs_.pop("users").groupby("id").agg(agg).reset_index()
for name, df in dfs_.items():
    # cols_rename = {label: "{}_{}".format(name, label) for label in ("id", "name", "description", "status_code")}
    cols_rename = {"id": "{}_id".format(name)}
    df.rename(columns=cols_rename, inplace=True)
    agg = aggs[name]
    agg[cols_rename["id"]] = ("count",)
    df = df.groupby("user_id").agg(agg)
    df_u =  df_u.merge(df, left_on="id", right_on="user_id", how="outer")
    #df_u.drop(columns="user_id", inplace=True)
df_u

In [ ]:
df_u.sort_values(("projects_id", "count"), ascending=False).head(20)

In [ ]:
df_u.id != 17

In [ ]:
df_u["fragsamples_id_count"] = df_u[("fragsamples_id", "count")]
df_u["projects_id_count"] = df_u[("projects_id", "count")]
df_u["matching_count_sum"] = df_u[("matching_count", "sum")]
df_u["reactions_id_count"] = df_u[("reactions_id", "count")]
df_u["annotations_count_sum"] = df_u[("annotations_count", "sum")]
sns.relplot(data=df_u[df_u.id != 17], x="fragsamples_id_count", y="projects_id_count", size="matching_count_sum", hue="annotations_count_sum", palette="ch:r=-.5,l=.75")

In [ ]:
df_u.loc[df_u["projects_id_count"] > 5].sort_values("fragsamples_id_count", ascending=False)

In [ ]:
agg = {"projects_id": ("nunique", "count"), "reactions_id": "nunique", "annotations_count": "sum"}
df_u.groupby("id").agg(agg)

In [ ]:
df_projects = dfs["projects"].merge(dfs["projects_info"], on="id", how="outer")

In [ ]:
df_projects.groupby("user_id").agg(["nunique", "count"]).frag_sample_id_y

In [ ]:
PROJECT_COLS = ("")
user_count = df_projects.reset_index().groupby("user_id").agg({"id": "count",  "matching_count": "sum"})#.head(20)
user_count.sort_values(by="matching_count", ascending=False).head()


In [ ]:
sns.displot(user_count.query("user_id!=17 and id<20"), x="id")

In [ ]:
sns.displot(user_count.query("user_id!=17"), x="id")

In [ ]:
sns.displot(user_count.query("user_id!=17 and matching_count > 0" ), x="matching_count")

In [ ]:
user_count.query("id == 0")

In [ ]:
df_fs = dfs["fragsamples"].merge(dfs["fragsamples_info"], on="id", how="outer")
df_fs


In [ ]:
df_fs.query("user_id == 92")

In [ ]:
df_fs_group = df_fs.groupby("user_id").agg({"name": "count", "annotations_count": "sum","ions_total": "sum"})
df_fs_group.sort_values("name", ascending=False)

In [ ]:
sns.displot(df_fs_group, x="name")

In [ ]:
sns.displot(df_fs_group, x="annotations_count")

In [ ]:
sns.displot(df_fs_group, x="ions_total")

In [ ]:
df_users = dfs["users"].reset_index().merge(df_fs_group, left_on="id", right_on="user_id", how="outer")
df_users = df_users.reset_index().merge(user_count, left_on="id", right_on="user_id", how="outer")
df_users.sort_values("name", ascending=False).head(20)

In [ ]:
df_users.sort_values("id_y", ascending=False).head(20)

In [ ]:
df_users

In [ ]:
fig, ax = plt.subplots()
#ax.set(xscale="log", yscale="log")
sns.scatterplot(x="id_y", y="matching_count", data=df_users.query("id_y < 400"),ax=ax)
plt.show()

In [ ]:
sns.relplot(x="ions_total", y="matching_count", data=df_users.query("ions_total < 60000"))